In [3]:
!source marshall_env/bin/activate

In [4]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from scipy.sparse import save_npz, load_npz
from scipy.sparse import csr_matrix, save_npz, load_npz
import dask.dataframe as dd
import gc

def batch_cosine_similarity(dtm, batch_size=1000):
    num_rows = dtm.shape[0]
    for start in range(0, num_rows, batch_size):
        end = min(start + batch_size, num_rows)
        yield cosine_similarity(dtm[start:end], dtm)

import psutil
def print_ram_usage(step = 0):
    process = psutil.Process()
    mem_info = process.memory_info()
    print(f"[{step}] RAM Usage: {mem_info.rss / (1024 ** 2):.2f} MB")

# # Load data (assuming data is in a CSV file, replace 'data.csv' with your actual file path)
# data = pd.read_csv('../int_data/Input_patent_matrix.csv')
data = pd.read_csv('../int_data/Input_patent_matrix_US.csv')
data

/users/xiet13/.local/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


,i,ind95us,description,street,city,county,state,region,full_address,names,pos2,max
0,US100001A,"[0.0, 826.0]",FOLLOWING EXACT DESCRIPTION ENABLE SKILLED ART...,[],"['Washington', 'Livingston']",['Sumter'],"['District of Columbia', 'Alabama']",[],"['Washington, District of Columbia', 'Livingst...","JOSEPH ARRINGTON, N. PETERS",-1,5690
1,US100003A,[0.0],APPARATUS MANUFACTURE SHEET UNITED PATENT GREA...,[],[],[],[],[],[],H. BESSEMER,-1,81450
2,US100004A,"[0.0, 105.0]",PHOTO PATENT ELATED FEBRUARY IMPROVEMENT HANGI...,[],"['Washington', 'Williamsport']",['Lycoming'],"['District of Columbia', 'Pennsylvania']",['Middle Atlantic Division'],"['Washington, DC', 'Williamsport, Lycoming Cou...","N. PETERS, JAMES M. BEUGLER",-1,2373
3,US100005A,[0.0],UNITED PATENT FEBRUARY IMPROVEMENT MANUFACTURE...,[],['New York'],[],['New York'],[],"['New York, NY']",MARCAR WAHRAM BEYLIKGY,-1,8220
4,US100007A,"[105.0, 336.0]",PHOTO UNITED WILLIAM PATENT FEBRUARY IMPROVEME...,[],['Altoona'],['Blair'],['Pennsylvania'],[],"['Altoona, Blair, Pennsylvania']","N. PETERS, WILLIAM A. BOYDEN, CHARLES J. MANN",-1,2729
...,...,...,...,...,...,...,...,...,...,...,...,...
158131,USRE7594E,[246.0],LOCK WORK ATTACHMENT APRIL PHOTO UNITED PATENT...,[],['New York'],['New York'],['New York'],[],"['New York, NY']",JOHN W. WILLIAMS,-1,2819
158132,USRE8666E,[0.0],SHEET TIME APRIL PHOTO SHEET PHOTO TIME G UNIT...,[],['Washington'],[],['District of Columbia'],[],"['Washington, D.C.']","LOUIS PAUL JU VET, N. PETERS",-1,8436
158133,USRE9467E,"[0.0, 0.0]",ROLLER ABSTRACTOR PHOTO C UNITED PATENT OFFICE...,[],"['Washington', 'Chicago']",['Cook'],"['District of Columbia', 'Illinois']",[],"['Washington, D.C.', 'Chicago, Cook County, Il...","BERNARD FRESE, N. PETERS",-1,4523
158134,USRE9656E,[636.0],SHEET CLOCK APRIL SHEET UNITED PATENT OFFICE C...,[],['Cheshire'],[],['Connecticut'],[],"['Cheshire, Connecticut']",ARTHUR E. HOTCHKISS,-1,19553


In [3]:
# dtm = load_npz('../int_data/dtm.npz')
dtm = load_npz('../int_data/US_proxy/dtm.npz') # Shape of dtm: (number of documents, size of the vocabulary)
dtm

<158136x30291 sparse matrix of type '<class 'numpy.int64'>'
	with 22198443 stored elements in Compressed Sparse Row format>

In [ ]:
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
from joblib import Parallel, delayed
from tqdm import tqdm

# Assume dtm is your sparse document-term matrix (e.g., csr_matrix)
n_rows = dtm.shape[0]

# Function to compute cosine similarity for a single row
def compute_row_cosine_similarity(row_idx, dtm):
    return cosine_similarity(dtm[row_idx], dtm, dense_output=False).toarray()[0]

# Parallel computation with a progress bar
results = Parallel(n_jobs=-1, prefer="threads")(
    delayed(compute_row_cosine_similarity)(i, dtm) for i in tqdm(range(n_rows), desc="Computing cosine similarity")
)

# Combine results into a sparse matrix
cosine_sim_matrix = csr_matrix(results)

Computing cosine similarity:   0%|          | 64/158136 [00:02<1:36:23, 27.33it/s]

In [ ]:
type(cosine_sim_matrix)

In [ ]:
import pickle

with open("../int_data/US_proxy/cosine_sim_matrix.pkl", "wb") as f:
    pickle.dump(cosine_sim_matrix, f)

In [4]:
!ls -lha ../int_data/US_proxy/cosine_sim_matrix.pkl

ls: cannot access ../int_data/US_proxy/cosine_sim_matrix.pkl: No such file or directory


In [ ]:
# calculate zero cells
# Given values
total_elements = cosine_sim_matrix.shape[0] * cosine_sim_matrix.shape[1]
non_zero_elements = cosine_sim_matrix.nnz
zero_elements = total_elements - non_zero_elements

zero_elements


- Explantions for zero cells despite all descriptions are present  
Descriptions may have a very low overlap in terms of vocabulary, leading to zero similarity.  
Descriptions might be very unique with little to no common words.

In [ ]:
# symmetry and upper triangle
from scipy.sparse import csr_matrix, triu

# Check if the matrix is square (necessary for symmetry)
if cosine_sim_matrix.shape[0] != cosine_sim_matrix.shape[1]:
    is_symmetric = False
else:
    # Check for symmetry
    is_symmetric = (cosine_sim_matrix != cosine_sim_matrix.transpose()).nnz == 0

if is_symmetric:
    # Extract the upper triangle
    upper_triangle = triu(cosine_sim_matrix)
    print("The matrix is symmetric.")
    # Optionally, convert to dense matrix for inspection (not recommended for large matrices)
    # print(upper_triangle.toarray())
else:
    print("The matrix is not symmetric.")

In [ ]:
upper_triangle

In [ ]:
# np.fill_diagonal(cosine_sim_matrix, 0)
cosine_sim_matrix.setdiag(0)

In [2]:
import pickle

# with open('../int_data/cosine_sim_matrix.pkl','wb') as f:
#     pickle.dump(cosine_sim_matrix,f) # 5.3 GB

with open('../int_data/US_proxy/cosine_sim_matrix_u.pkl','wb') as f:
    pickle.dump(upper_triangle,f)
    
with open('../int_data/US_proxy/cosine_sim_matrix_u.pkl','rb') as f:
    upper_triangle = pickle.load(f) # 22,120 unique patent-inventor pairs
upper_triangle

FileNotFoundError: [Errno 2] No such file or directory: '../int_data/US_proxy/cosine_sim_matrix_u.pkl'